# Problem introduction

An international seller of luxury furnishing wants to:
1. launch a marketing campaign in Belgium 
2. create a few stores in Belgium (if it estimates the market is not packed yet of similar stores)

For that purpose, it wants first to identify on a map where its potential customers are clustered (i.e. spot the highest number of potential customer per postal code).
The potential customers share the following features:
1. Age: 18-44 years
2. High propensity to buy luxury furniture’s assessed based on  housing price 
3. Owner of the housing (these are assumed to be more likely to buy luxury furniture’s)

Second, for the launch of its stores, it wants to avoid locations where similar stores already exist. It will accept to create a store in a location only if the number of similar stores per potential customer (within a certain radius) is below a certain threshold.

In this project, we will show on a map where its potential clients are mostly concentrated. To this end, we will color postal codes based on a scale of low number of potential clients (Dark Blue) to high number of potential clients (Dark Red). Postal codes for which the assessment could not be made will be marked in Grey.


## 1. Import data and data wrangling

In [3]:
import pandas as pd
df_PostalCode_LongLat = pd.read_csv('Geospatial_Coordinates_Belgium_UTF.csv')
df_PostalCode_LongLat.rename(columns={'Label':'Commune'},inplace=True)
df_PostalCode_LongLat['Commune']=df_PostalCode_LongLat['Commune'].str.lower()
df_PostalCode_LongLat.head()

,Postal Code,Commune,Longitude,Latitude
0,1000,bruxelles,4.351697,50.846557
1,1020,laeken,4.348713,50.883392
2,1030,schaerbeek,4.373712,50.867604
3,1040,etterbeek,4.389510,50.836851
4,1050,ixelles,4.381571,50.822285


In [4]:
df_age = pd.read_csv('Brussels_Age_18_44_Clean.csv')
df_age.rename(columns={'Territoire':'Commune'},inplace=True)
df_age.drop(["Population totale (Nombre d'habitants)"], axis = 1, inplace=True)
df_age.head()
#condition=df_age['Commune']=='Saint-Josse-ten-Noode'
#df_age[condition]
df_age['Commune']=df_age['Commune'].str.lower()
df_age

,Commune,Population 18-44
0,anderlecht,45852
1,auderghem,12631
2,berchem-sainte-agathe,8577
3,bruxelles,77383
4,etterbeek,23116
5,evere,14781
6,forest,22915
7,ganshoren,8503
8,ixelles,44825
9,jette,19798


In [5]:
df_owner = pd.read_csv('logement occupé par proprio_2011_Clean.csv')
df_owner.drop(['Belgique','RÃ©gion','Province','Arrondissement'], axis = 1, inplace=True)
df_owner.rename(columns={'Logements occupÃ©s par leur propriÃ©taire':'N_Proprio'},inplace=True)
df_owner['N_Proprio']=round(df_owner['N_Proprio']*df_owner['Nombre de logements'],0)
df_owner.head()
#condition=df_owner['Commune']=='Forest'
#df_owner[condition]

,Commune,Nombre de logements,N_Proprio
0,Aartselaar,5741,4587.0
1,Anvers,213778,111378.0
2,Boechout,4839,3760.0
3,Boom,7187,4858.0
4,Borsbeek,4507,2596.0


In [6]:
df_owner.drop(['Nombre de logements'], axis = 1, inplace=True)

In [7]:
df_owner['Commune']=df_owner['Commune'].str.lower()
df_owner.head()

,Commune,N_Proprio
0,aartselaar,4587.0
1,anvers,111378.0
2,boechout,3760.0
3,boom,4858.0
4,borsbeek,2596.0


In [8]:
df_income = pd.read_csv('Income.csv')
df_income.drop(['<20KEUR','20-30KEUR','30-40KEUR'], axis = 1, inplace=True)
df_income.head()

,Commune,>40EUR
0,Région flamande,776498.0
1,Région wallonne,342413.0
2,Communauté germanophone,8083.0
3,Région de Bruxelles-Capitale,59673.0
4,Province d’Anvers,204575.0


In [9]:
df_income.rename(columns={'>40EUR':'>40KEUR'},inplace=True)
df_income['Commune']=df_income['Commune'].str.lower()
df_income.head()
x=df_income['>40KEUR'].mean(axis=0)
df_income.shape

(646, 2)

In [10]:
df_income.dropna(subset=[">40KEUR"], axis=0, inplace=True)
df_income.shape

(642, 2)

In [11]:
df_PriceImmo = pd.read_csv('immo_by_municipality_2010-2019.csv')
df_PriceImmo.rename(columns={'CD_REFNIS_FR':'Commune'},inplace=True)
df_PriceImmo.head()
#df_PriceImmo['Commune_']=df_PriceImmo['Commune']
#df_PriceImmo['Commune']=df_PriceImmo['Commune_']
df_PriceImmo['Commune']=df_PriceImmo['Commune'].str.lower()

#for i in range(145,146,1):
#    a=df_PriceImmo['Commune_'][i][0]
#    b=df_PriceImmo['Commune_'][i][1:20].lower()
#    df_PriceImmo['Commune'][i]=a+b
#df_PriceImmo['Commune'][i]

#condition=df_PriceImmo['Commune']=='BERCHEM-SAINTE-AGATHE'
#df_PriceImmo[condition]
df_PriceImmo.head()

,CD_YEAR,CD_TYPE_FR,CD_REFNIS,CD_REFNIS_NL,Commune,MS_TOTAL_TRANSACTIONS,MS_TOTAL_PRICE_
0,2015,maisons d'habitation,44001,AALTER,aalter,57,15597195
1,2015,villas,44001,AALTER,aalter,36,13349740
2,2015,appartements,44001,AALTER,aalter,38,8263800
3,2015,maisons d'habitation,24001,AARSCHOT,aarschot,128,26286689
4,2015,villas,24001,AARSCHOT,aarschot,45,12231500


In [12]:
df_PriceImmo.head()
df_PriceImmo.drop(['CD_YEAR','CD_TYPE_FR','CD_REFNIS','CD_REFNIS_NL'], axis = 1, inplace=True)

In [13]:
df_PriceIm=df_PriceImmo
df_PriceIm = df_PriceIm.groupby('Commune', axis=0,as_index= False).sum()
df_PriceIm['AvgPrice']=round(df_PriceIm['MS_TOTAL_PRICE_']/df_PriceIm['MS_TOTAL_TRANSACTIONS'],0)
df_PriceIm.head()

,Commune,MS_TOTAL_TRANSACTIONS,MS_TOTAL_PRICE_,AvgPrice
0,aalter,131,37210735,284051.0
1,aarschot,223,50507244,226490.0
2,aartselaar,186,54332475,292110.0
3,affligem,101,24960650,247135.0
4,aiseau-presles,95,11997400,126288.0


In [14]:
df_PriceIm.drop(['MS_TOTAL_TRANSACTIONS','MS_TOTAL_PRICE_'], axis = 1, inplace=True)

In [15]:
df = pd.merge(df_age,df_PostalCode_LongLat)
df=pd.merge(df,df_owner)
df=pd.merge(df,df_income)
df=pd.merge(df,df_PriceIm)
df.head(20)

,Commune,Population 18-44,Postal Code,Longitude,Latitude,N_Proprio,>40KEUR,AvgPrice
0,anderlecht,45852,1070,4.312340,50.838141,16960.0,5061.0,189660.0
1,auderghem,12631,1160,4.433139,50.815657,7253.0,2711.0,336699.0
2,berchem-sainte-agathe,8577,1082,4.292702,50.863984,4799.0,1837.0,199323.0
3,bruxelles,77383,1000,4.351697,50.846557,20535.0,6996.0,279323.0
4,etterbeek,23116,1040,4.389510,50.836851,6908.0,2068.0,306969.0
5,evere,14781,1140,4.402160,50.870452,6693.0,2359.0,220431.0
6,forest,22915,1190,4.317751,50.809143,9160.0,2868.0,240603.0
7,forest,22915,7910,3.537315,50.673615,9160.0,2868.0,240603.0
8,ganshoren,8503,1083,4.317510,50.871240,4926.0,1575.0,188926.0
9,ixelles,44825,1050,4.381571,50.822285,12147.0,3672.0,319833.0


## 2. Data pre-processing

In [33]:
#Assign to X explanatory variables
X=df[['Population 18-44','N_Proprio','>40KEUR','AvgPrice']]

#Normalise data
from sklearn.preprocessing import StandardScaler
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset


/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


array([[-0.15904517, -0.10522313, -0.15717784, -0.3471917 ],
       [-0.72842208, -0.4850821 , -0.25243714,  0.9840296 ],
       [-0.95287307, -0.97389904, -0.78273414, -0.91904501],
       [-0.99212708, -0.88047803, -0.62680012,  1.15600119],
       [-0.9569701 , -0.94860167, -0.94170189, -1.06307502],
       [-0.33161886,  0.02823545,  0.1188921 , -0.96499553],
       [-0.65041235, -0.07474676,  0.17531958,  2.20536863],
       [-0.60938665, -0.59662956, -0.46601213, -0.62663514],
       [ 0.17624718,  1.55444386,  1.88331276,  1.9004079 ],
       [ 1.79634705,  1.60842486,  1.22195839, -0.37227955],
       [-0.72249799, -1.37407558, -1.5830336 , -0.59897063],
       [-0.94717044, -1.23802751, -1.27359257, -1.13710564],
       [ 0.65964175,  0.56784633,  0.20505019, -1.1097459 ],
       [ 1.11086907,  1.44847376,  1.17341862, -1.05290689],
       [-0.25615594,  0.26866009,  0.40224301,  0.78965766],
       [-0.15904517, -0.10522313, -0.15717784, -0.3471917 ],
       [-0.14791675, -0.

## 3. Modeling
Let's run our model and group locations into five clusters.

In [35]:
from sklearn.cluster import KMeans 
num_clusters = 3

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[0 2 0 2 0 0 2 0 2 1 0 0 0 1 2 0 0 1 0 2]


In [36]:
df["Labels"] = labels
df_X=df[['Population 18-44','N_Proprio','>40KEUR','AvgPrice','Labels']]

In [37]:
df_X.groupby('Labels').mean().round(0)

,Population 18-44,N_Proprio,>40KEUR,AvgPrice
Labels,,,,
0,18617.0,6909.0,2130.0,222683.0
1,60489.0,18419.0,5733.0,235925.0
2,21583.0,10418.0,3652.0,359338.0


In [20]:
df.head()

,Commune,Population 18-44,Postal Code,Longitude,Latitude,N_Proprio,>40KEUR,AvgPrice,Labels
0,anderlecht,45852,1070,4.312340,50.838141,16960.0,5061.0,189660.0,2
1,auderghem,12631,1160,4.433139,50.815657,7253.0,2711.0,336699.0,0
2,berchem-sainte-agathe,8577,1082,4.292702,50.863984,4799.0,1837.0,199323.0,1
3,bruxelles,77383,1000,4.351697,50.846557,20535.0,6996.0,279323.0,2
4,etterbeek,23116,1040,4.389510,50.836851,6908.0,2068.0,306969.0,1


### Cluster interpretation
- Label_0: Relatively low number of young people and do not seem really wealthy: slighlty higher than Label 2 for  # of young people, lowest # of salary >40kEUR and lowest avg price of houses 
- Label_1: Relatively high number of young people and relatively wealthy (based on avg price of houses)): 1st for young people, first for # owners , first for # of salary >40kEUR but do not seem the richest neighboor
- Label_2: Wealthy (based on avgprice of houses and # of >40kEUR salary) but few young people: just below Label 0 for  # of young people but higher for # owners , second for # of salary >40kEUR and first for avg price house

Based on these observations, it is difficult to make a recommendation between Label_1 and Label_2. On one hand Label_2 seems to include the most wealthy people, but on the other hand, there is a lower amount of people earning >40k than for Label_1... On top of that, the population between 18 and 44 seem the lowest for this label.

All in all, we still recommand to target Label_2 neighbourhood - as they seem to be the wealthiest and have still a significant amount of population earning >40kEUR.Second, we recommand Label_1 and thirs Label_0

In [21]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.7                |           py36_0         869 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         869 KB

The following packages will be UPDATED:

  conda                                        4.6.4-py36_0 --> 4.6.7-py36_0



conda-4.6.7          | 869 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [38]:
import folium # plotting library
latitude=50.822285
longitude=4.381571

colors_list = ['grey', 'yellow', 'red']

venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

#for i,commune, lat, lng, label , color in zip(df.index,df.Commune, df.Latitude, df.Longitude, df.Labels,colors_list):
for i,lat, lng, in zip (df.index,df.Latitude, df.Longitude):
        folium.features.CircleMarker(
            [lat,lng],
            radius=50,
            color=colors_list[df.Labels[i]],
            popup=df.Commune[i],
            fill = True,
            fill_color=colors_list[df.Labels[i]],
            fill_opacity=0.6
        ).add_to(venues_map)
venues_map

# 4. Import number of venues per neighborhood


## Test querry

In [23]:
#test query
CLIENT_ID = '1SEG22KQUVDW0BIUOJ10GSAM1EOG1DBBFRZYWQCNUSKMTZVP' # your Foursquare ID
CLIENT_SECRET = 'ZDVVBSJMWWEWZAGPIDK4ANIVP2O0QGGI0JPGDV0CALUREZ2N' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
search_query = 'furniture'
latitude = df.Latitude[1] 
longitude = df.Longitude[1]
radius=5000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    

In [24]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import requests
results = requests.get(url).json()
venues=results['response']['venues']

In [25]:
##### tranform venues into a dataframe and test the count of number of 
dataframe = json_normalize(venues)

print('there are',dataframe['categories'].count(),'furtnitures stores in a radius of ',radius, 'meters')

there are 2 furtnitures stores in a radius of  5000 meters


## perform qerry for each postal code

In [40]:
import numpy as np
n=20
search_query = 'furniture'
N_proxy_competitors=np.zeros(n)
df['N_Comp']=0


for i,lat, lng, in zip (df.index[0:30],df.Latitude, df.Longitude):
    try:
        latitude = df.Latitude[i]
        longitude = df.Longitude[i]
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
        venues=results['response']['venues']
        dataframe = json_normalize(venues)
        N_proxy_competitors[i]=dataframe['categories'].count()
        df["N_Comp"].loc[i]=dataframe['categories'].count()
    
    except: N_proxy_competitors[i]=0

df.sort_values(by='N_Comp', ascending=True, axis=0, inplace=True)
df.head(20)
    


,Commune,Population 18-44,Postal Code,Longitude,Latitude,N_Proprio,>40KEUR,AvgPrice,Labels,N_Comp
7,forest,22915,7910,3.537315,50.673615,9160.0,2868.0,240603.0,0,0
1,auderghem,12631,1160,4.433139,50.815657,7253.0,2711.0,336699.0,2,2
2,berchem-sainte-agathe,8577,1082,4.292702,50.863984,4799.0,1837.0,199323.0,0,2
17,watermael-boitsfort,7868,1170,4.415818,50.799394,5268.0,2094.0,349113.0,2,2
8,ganshoren,8503,1083,4.317510,50.871240,4926.0,1575.0,188926.0,0,2
10,jette,19798,1090,4.326090,50.877763,9830.0,3323.0,196006.0,0,2
19,woluwe-saint-pierre,14040,1150,4.443297,50.829243,9313.0,3416.0,424863.0,2,3
5,evere,14781,1140,4.402160,50.870452,6693.0,2359.0,220431.0,0,3
14,saint-josse-ten-noode,12738,1210,4.372336,50.853074,2790.0,518.0,222428.0,0,4
16,uccle,28971,1180,4.337235,50.801820,17492.0,6231.0,402849.0,2,4


# Conclusion

Based on the number of competitors in a radius of 5km, Auderghem seems a good candidate to open a luxury furnishing store as there are few competitors around. 
On top of that, Auderghem belongs to the Label_2 cluster of neihgbours which is the cluster with the highest potential to sell luxury furnishing (cf. Section3.).

In [28]:
import folium # plotting library
from folium import IFrame
latitude=50.822285
longitude=4.381571

colors_list = ['grey', 'yellow', 'red']

venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel



#for i,commune, lat, lng, label , color in zip(df.index,df.Commune, df.Latitude, df.Longitude, df.Labels,colors_list):
for i,lat, lng, in zip (df.index,df.Latitude, df.Longitude):
        folium.features.CircleMarker(
            [lat,lng],
            radius=50,
            color=colors_list[df.Labels[i]],
            popup=df.Commune[i],
            fill = True,
            fill_color=colors_list[df.Labels[i]],
            fill_opacity=0.6
        ).add_to(venues_map)
        
venues_map